In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [86]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_race`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_4864/3685221015.py:10: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



In [87]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,31,E OCON,OCO,ocon,Alpine,2293D1,alpine,Esteban,Ocon,Esteban Ocon,https://www.formula1.com/content/dam/fom-websi...,FRA,18,R,8,NaT,NaT,NaT,NaT,Collision damage,0,2023,United States Grand Prix
1,10,P GASLY,GAS,gasly,Alpine,2293D1,alpine,Pierre,Gasly,Pierre Gasly,https://www.formula1.com/content/dam/fom-websi...,FRA,6,6,7,NaT,NaT,NaT,0 days 00:00:47.996000,Finished,8,2023,United States Grand Prix
2,14,F ALONSO,ALO,alonso,Aston Martin,358C75,aston_martin,Fernando,Alonso,Fernando Alonso,https://www.formula1.com/content/dam/fom-websi...,ESP,16,R,0,NaT,NaT,NaT,NaT,Undertray,0,2023,United States Grand Prix
3,18,L STROLL,STR,stroll,Aston Martin,358C75,aston_martin,Lance,Stroll,Lance Stroll,https://www.formula1.com/content/dam/fom-websi...,CAN,7,7,0,NaT,NaT,NaT,0 days 00:00:48.696000,Finished,6,2023,United States Grand Prix
4,11,S PEREZ,PER,perez,Red Bull Racing,3671C6,red_bull,Sergio,Perez,Sergio Perez,https://www.formula1.com/content/dam/fom-websi...,MEX,4,4,9,NaT,NaT,NaT,0 days 00:00:18.460000,Finished,12,2023,United States Grand Prix


In [88]:
#Get the counts by final position in race

In [89]:
grouped = GP.groupby(['FullName','Position'])["GP"].agg('count').reset_index()
grouped.columns=["Driver","Position","Count"]
grouped["Rank"] = grouped.groupby(['Position'])["Count"].rank(method="dense", ascending=False)
grouped=grouped.sort_values(by=["Position","Count"],ascending=[True, False])

In [90]:
#Plot 1 lap at a time

In [91]:
grouped2=grouped[(grouped['Position'].isin([1]))&(grouped['Count']>0)]

In [92]:
position=grouped2["Position"].iloc[0]

In [123]:
fig_lap=px.bar(grouped2, 
           x="Driver", 
           y="Count", 
        #    facet_col='Position', facet_col_wrap=1, 
           height=1000,
           width=1800,
            color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 }
        )
fig_lap.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title="Race Result Counts",
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_lap.update_traces(marker_color='#246DF8')

In [85]:
#Plot the top in each final position

In [54]:
grouped_top=grouped[grouped["Rank"]==1]
grouped_top=grouped_top[grouped_top["Position"]!=0]
grouped_top

,Driver,Position,Count,Rank
4134,Lewis Hamilton,1,104,1.0
4135,Lewis Hamilton,2,56,1.0
4011,Kimi Räikkönen,3,45,1.0
6542,Sebastian Vettel,4,34,1.0
1902,Fernando Alonso,5,29,1.0
1864,Felipe Massa,6,27,1.0
1904,Fernando Alonso,7,28,1.0
891,Carlos Sainz,8,21,1.0
1867,Felipe Massa,9,26,1.0
6430,Rubens Barrichello,10,20,1.0


In [124]:
fig_top=px.bar(grouped_top, 
           x="Position", 
           y="Count", 
        #    color='Driver',
           height=1000,
           width=1800,
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 },
            hover_data=["Driver"]
        )

fig_top.update_layout(
    title="Leaders By Final Race Position",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_top.update_traces(marker_color='#246DF8')

In [94]:
#By Grid Position

In [95]:
grouped_grid = GP.groupby(['FullName','GridPosition'])["GP"].agg('count').reset_index()
grouped_grid.columns=["Driver","GridPosition","Count"]
grouped_grid["Rank"] = grouped_grid.groupby(['GridPosition'])["Count"].rank(method="dense", ascending=False)
grouped_grid=grouped_grid.sort_values(by=["GridPosition","Count"],ascending=[True, False])

In [96]:
#Plot 1 lap at a time

In [101]:
grouped2_grid=grouped_grid[(grouped_grid['GridPosition'].isin([1]))&(grouped_grid['Count']>0)]

In [102]:
position=grouped2_grid["GridPosition"].iloc[0]

In [125]:
fig_lap_grid_pos=px.bar(grouped2_grid, 
           x="Driver", 
           y="Count", 
        #    facet_col='Position', facet_col_wrap=1, 
           height=1000,
           width=1800,
            color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 }
        )
fig_lap_grid_pos.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title="Race Grid Position Counts",
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_lap_grid_pos.update_traces(marker_color='#246DF8')

In [104]:
#Plot the top grid positions

In [105]:
grouped_top_grid=grouped_grid[grouped_grid["Rank"]==1]
grouped_top_grid=grouped_top_grid[grouped_top_grid["GridPosition"]!=0]

In [126]:
fig_top_grid=px.bar(grouped_top_grid, 
           x="GridPosition", 
           y="Count", 
        #    color='Driver',
           height=1000,
           width=1800,
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 },
            hover_data=["Driver"]
        )

fig_top_grid.update_layout(
    title="Leaders By Final Grid Position",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_top_grid.update_traces(marker_color='#246DF8')

In [107]:
#Qualifying

In [108]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_qualifying`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_4864/832157024.py:10: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



In [109]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,21,0,ALO,0,Minardi,0,0,Fernando,Alonso,Fernando Alonso,0,0,17,17,17,0 days 00:01:13.991000,0,0,0,0,0,2001,United States Grand Prix
1,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,12,12,12,0 days 00:01:20.543000,0,0,0,0,0,1995,Hungarian Grand Prix
2,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,13,13,13,0 days 00:01:57.167000,0,0,0,0,0,1995,Argentine Grand Prix
3,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,15,15,15,0,0,0,0,0,0,1995,Australian Grand Prix
4,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,16,16,16,0 days 00:01:49.302000,0,0,0,0,0,1995,German Grand Prix


In [110]:
#Get the counts by final position in Qualy

In [111]:
grouped = GP.groupby(['FullName','Position'])["GP"].agg('count').reset_index()
grouped.columns=["Driver","Position","Count"]
grouped["Rank"] = grouped.groupby(['Position'])["Count"].rank(method="dense", ascending=False)
grouped=grouped.sort_values(by=["Position","Count"],ascending=[True, False])

In [112]:
#Plot 1 lap at a time

In [119]:
grouped2=grouped[(grouped['Position'].isin([1]))&(grouped['Count']>0)]

In [120]:
position=grouped2["Position"].iloc[0]

In [127]:
fig_lap=px.bar(grouped2, 
           x="Driver", 
           y="Count", 
        #    facet_col='Position', facet_col_wrap=1, 
           height=1000,
           width=1800,
            color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 }
        )
fig_lap.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title="Qualifying Result Counts",
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_lap.update_traces(marker_color='#246DF8')

In [116]:
#Plot the top in each final position

In [117]:
grouped_top=grouped[grouped["Rank"]==1]
grouped_top=grouped_top[grouped_top["Position"]!=0]
grouped_top

,Driver,Position,Count,Rank
926,Lewis Hamilton,1,107,1.0
927,Lewis Hamilton,2,75,1.0
1586,Sebastian Vettel,3,47,1.0
872,Kimi Räikkönen,4,37,1.0
431,Fernando Alonso,5,38,1.0
432,Fernando Alonso,6,29,1.0
874,Kimi Räikkönen,6,29,1.0
433,Fernando Alonso,7,26,1.0
434,Fernando Alonso,8,26,1.0
435,Fernando Alonso,9,23,1.0


In [128]:
fig_top=px.bar(grouped_top, 
           x="Position", 
           y="Count", 
        #    color='Driver',
           height=1000,
           width=1800,
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 },
            hover_data=["Driver"]
        )

fig_top.update_layout(
    title="Leaders By Final Qualifying Position",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_top.update_traces(marker_color='#246DF8')

In [ ]:
#Final Position vs. Grid Position

In [138]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_race`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_4864/3685221015.py:10: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



In [139]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP_2023=GP[GP['Year']==2023]
GP_2023.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
50,31,E OCON,OCO,ocon,Alpine,2293D1,alpine,Esteban,Ocon,Esteban Ocon,https://www.formula1.com/content/dam/fom-websi...,FRA,18,R,8,NaT,NaT,NaT,NaT,Collision damage,0,2023,United States Grand Prix
51,10,P GASLY,GAS,gasly,Alpine,2293D1,alpine,Pierre,Gasly,Pierre Gasly,https://www.formula1.com/content/dam/fom-websi...,FRA,6,6,7,NaT,NaT,NaT,0 days 00:00:47.996000,Finished,8,2023,United States Grand Prix
52,14,F ALONSO,ALO,alonso,Aston Martin,358C75,aston_martin,Fernando,Alonso,Fernando Alonso,https://www.formula1.com/content/dam/fom-websi...,ESP,16,R,0,NaT,NaT,NaT,NaT,Undertray,0,2023,United States Grand Prix
53,18,L STROLL,STR,stroll,Aston Martin,358C75,aston_martin,Lance,Stroll,Lance Stroll,https://www.formula1.com/content/dam/fom-websi...,CAN,7,7,0,NaT,NaT,NaT,0 days 00:00:48.696000,Finished,6,2023,United States Grand Prix
54,11,S PEREZ,PER,perez,Red Bull Racing,3671C6,red_bull,Sergio,Perez,Sergio Perez,https://www.formula1.com/content/dam/fom-websi...,MEX,4,4,9,NaT,NaT,NaT,0 days 00:00:18.460000,Finished,12,2023,United States Grand Prix


In [154]:
fig_scatter_grid_vs_final=px.scatter(GP_2023, 
           x="GridPosition", 
           y="Position", 
           color='Abbreviation',
           facet_col='GP', facet_col_wrap=1, 
           text='Abbreviation',
           facet_row_spacing=0.02, # default is 0.07 when facet_col_wrap is used
           facet_col_spacing=0.04,
           height=4000,
           width=1800,
           range_y=[0,25],
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 },
            hover_data=["Abbreviation"]
        )

fig_scatter_grid_vs_final.update_layout(
    title="Final Position vs. Grid Position",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter_grid_vs_final.update_traces(textposition='middle right', marker=dict(size=10))



In [ ]:
#Driver Points for a Season

In [186]:
GP_2023.groupby(['Abbreviation'])["Points"].agg('sum').reset_index()

,Abbreviation,Points
0,ALB,25
1,ALO,198
2,BOT,10
3,DEV,0
4,GAS,54
5,HAM,217
6,HUL,6
7,LAW,2
8,LEC,185
9,MAG,3


In [204]:
fig_driver_pts=px.histogram(GP_2023.groupby(['FullName'])["Points"].agg('sum').reset_index().sort_values(by=["Points"],ascending=False), 
           x="FullName", 
           y="Points", 
           color='FullName',
           text_auto=True,
           height=1000,
           width=1800,
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 },
            hover_data=["FullName"]
        )

fig_driver_pts.update_layout(
    title="Driver Points",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_driver_pts.update_traces(marker_color='#246DF8')
# fig_driver_pts.update_traces(textfont_size=12, textposition="outside")


In [ ]:
#Team Points for a Season

In [193]:
fig_team_pts=px.histogram(GP_2023.groupby(['TeamName'])["Points"].agg('sum').reset_index().sort_values(by=["Points"],ascending=False), 
           x="TeamName", 
           y="Points", 
           color='TeamName',
           text_auto=True,
           height=1000,
           width=1800,
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 },
            hover_data=["TeamName"]
        )

fig_team_pts.update_layout(
    title="Team Points",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_team_pts.update_traces(textfont_size=12, textposition="outside")


In [ ]:
#Driver Points All Time

In [201]:
fig_driver_pts_all_time=px.histogram(GP.groupby(['FullName'])["Points"].agg('sum').reset_index().sort_values(by=["Points"],ascending=False), 
           x="FullName", 
           y="Points", 
           color='FullName',
        #    text_auto=True,
           height=1000,
           width=1800,
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 },
            hover_data=["FullName"]
        )

fig_driver_pts_all_time.update_layout(
    title="Driver Points",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=10)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_driver_pts_all_time.update_traces(marker_color='#246DF8')
# fig_driver_pts_all_time.update_traces(textfont_size=12, textposition="outside")


In [ ]:
#Team Points All Time

In [195]:
fig_team_pts_all_time=px.histogram(GP.groupby(['TeamName'])["Points"].agg('sum').reset_index().sort_values(by=["Points"],ascending=False), 
           x="TeamName", 
           y="Points", 
           color='TeamName',
        #    text_auto=True,
           height=1000,
           width=1800,
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 },
            hover_data=["TeamName"]
        )

fig_team_pts_all_time.update_layout(
    title="Team Points",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
# fig_team_pts_all_time.update_traces(textfont_size=12, textposition="outside")
